In [ ]:
import pyodbc
from datetime import datetime, timezone, timedelta
import csv

In [37]:
class SaveToSQLServerPipeLine:

    def __init__(self):
        self.conn = pyodbc.connect(
            'DRIVER={SQL Server};'
            'SERVER=DinhNamNguyen\DinhNam;'  # Hoặc là tên server SQL của bạn
            'DATABASE=vnwork_stg;'
            'Trusted_Connection=yes;'
        )

        ## Create cursor, used to execute commands
        self.cur = self.conn.cursor()

        ## Create jobs table if none exists
        self.cur.execute("""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='jobs' AND xtype='U')
        CREATE TABLE jobs(
            id VARCHAR(500) NOT NULL PRIMARY KEY, 
            url VARCHAR(1000),
            title NVARCHAR(500),
            company_name NVARCHAR(500),
            company_url VARCHAR(1000),
            time_update DATETIME,
            time_expire DATETIME,
            salary DECIMAL(10,2),
            exp INT,
            job_level NVARCHAR(500),
            group_job NVARCHAR(500),
            job_type NVARCHAR(500),
            benefit TEXT,
            job_des TEXT,
            job_req TEXT,
            city NVARCHAR(500),
            address TEXT,
            web NVARCHAR(500)         
        )
        """)
        self.conn.commit()

        self.cur.execute("""
        IF NOT EXISTS (
            SELECT * 
            FROM sys.indexes 
            WHERE name='IX_jobs_url' AND object_id=OBJECT_ID('jobs')
        )
        BEGIN
            CREATE INDEX IX_jobs_url ON jobs(url);
        END
        """)
        self.conn.commit()
    
    def process_item(self, item):
        # Define insert statement
        self.cur.execute("""
            INSERT INTO jobs (id, url, title, company_name, company_url, time_update, time_expire, salary, exp, job_level, group_job, job_type, benefit, job_des, job_req, city, address, web)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        """, (
            item["id"],
            item["url"],
            item["title"],
            item["company_name"],
            item["company_url"],
            item["time_update"],
            item["time_expire"],
            item["salary"],
            item["exp"],
            item["job_level"],
            item["group_job"],
            item["job_type"],
            item["benefit"],
            item["job_des"],
            item["job_req"],
            item["city"],
            item["address"],
            item["web"]
        ))

        # Commit để lưu dữ liệu vào database
        self.conn.commit()
        return item



    def close_connection(self):
        ## Close cursor & connection to database 
        self.cur.close()
        self.conn.close()

    def get_links_from_database(self):
        # Sử dụng kết nối hiện tại (self.conn) thay vì mở kết nối mới
        cursor = self.conn.cursor()

        query = """
        SELECT url 
        FROM jobs WITH (INDEX(IX_jobs_url))
        """
        cursor.execute(query)

        # Lấy tất cả các URL từ kết quả và lưu vào danh sách
        list_url_in_db = [row[0] for row in cursor.fetchall()]

        cursor.close()  # Đóng cursor sau khi sử dụng
        return list_url_in_db
    
    def get_count_url(self):
        cursor = self.conn.cursor()

        query = """
        SELECT COUNT(*) 
    FROM jobs WITH (INDEX(IX_jobs_url))

        """
        cursor.execute(query)

        # Lấy số lượng bản ghi
        record_count = cursor.fetchone()[0]

        cursor.close()

        return record_count
    

    

In [38]:
import requests
import json
import pandas as pd
import time

url = "https://ms.vietnamworks.com/job-search/v1.0/search"

page = 0

payload = {
    "userId": 0,
    "query": "",
    "filter": [],
    "ranges": [],
    "order": [],
    "hitsPerPage": 50,
    "page": page,
    "retrieveFields": [
        "address",
        "benefits",
        "jobTitle",
        "salaryMax",
        "isSalaryVisible",
        "jobLevelVI",
        "isShowLogo",
        "salaryMin",
        "companyLogo",
        "userId",
        "jobLevel",
        "jobLevelId",
        "jobId",
        "jobUrl",
        "companyId",
        "approvedOn",
        "isAnonymous",
        "alias",
        "expiredOn",
        "industries",
        "workingLocations",
        "services",
        "companyName",
        "salary",
        "onlineOn",
        "simpleServices",
        "visibilityDisplay",
        "isShowLogoInSearch",
        "priorityOrder",
        "skills",
        "profilePublishedSiteMask",
        "jobDescription",
        "jobRequirement",
        "prettySalary",
        "requiredCoverLetter",
        "languageSelectedVI",
        "languageSelected",
        "languageSelectedId",
        "yearsOfExperience"
    ]
}

headers = {
  'Connection': 'keep-alive',
  'Origin': 'https://www.vietnamworks.com',
  'Referer': 'https://www.vietnamworks.com/',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-site',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0',
  'accept': '*/*',
  'accept-language': 'vi',
  'content-type': 'application/json',
  'sec-ch-ua': '"Not)A;Brand";v="99", "Microsoft Edge";v="127", "Chromium";v="127"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'x-source': 'Page-Container'
}



In [39]:
jobs_list = []
pipeline = SaveToSQLServerPipeLine()
url_list_in_db = pipeline.get_links_from_database()

In [40]:
#168
while page < 168:
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
    response_data = response.json()

    time.sleep(3)

    for job in response_data.get('data', []):
        job_url = job.get('jobUrl', 'N/A')

        if job_url in url_list_in_db:
            continue
        else:
            count = pipeline.get_count_url()
            job_id = "VNW" + str(count)
            
            title = job.get('jobTitle')

            groupJobFunctionsV3 = job.get('groupJobFunctionsV3',[])
            group_job_name = groupJobFunctionsV3['groupJobFunctionV3NameVI']

            jobFunctionsV3 = job.get('jobFunctionsV3',[])
            job_type = jobFunctionsV3['jobFunctionV3NameVI']

            job_url = job.get('jobUrl')

            job_level = job.get('jobLevelVI', 'N/A')

            time_update = job.get('approvedOn', 'N/A')
            time_expire = job.get('expiredOn', 'N/A')

            if time_update:
                time_update = datetime.strptime(time_update[:10], "%Y-%m-%d")

            if time_expire:
                time_expire = datetime.strptime(time_expire[:10], "%Y-%m-%d")


            pretty_salary = job.get('salary','0')
            try:
                if pretty_salary:
                    pretty_salary = int(pretty_salary)
                    if pretty_salary > 1000000:
                        pretty_salary = pretty_salary/25000
                else:
                    pretty_salary = 0
            except ValueError:
                pretty_salary = 0

            # Xử lý workingLocations nếu nó là danh sách
            city_func = job.get('workingLocations', [])
            if city_func:
                # Giả sử bạn chỉ cần thông tin từ phần tử đầu tiên
                city_info = city_func[0]  # Lấy phần tử đầu tiên
                city = city_info.get('cityName', 'Not Show')
                address = city_info.get('address', 'Not Show')

            company = job.get('companyName')
            company_url = job.get('companyLogo','Not Show')

            job_description = job.get('jobDescription', 'N/A')
            job_requirement = job.get('jobRequirement', 'N/A')

            years_of_exp = job.get('yearsOfExperience','0')
            try:
                if years_of_exp:
                    years_of_exp = round(float(years_of_exp))
            except ValueError:
                years_of_exp = 0

            benefit_info = job.get('benefits', [])

            if benefit_info:
                benefit_names = ','.join(benefit['benefitName'] for benefit in benefit_info)
                    

        job_data ={
            'id': job_id,
            'url': job_url,
            'title': title,
            'company_name' : company,
            'company_url' : company_url,
            'time_update': time_update,
            'time_expire' : time_expire,
            'salary': pretty_salary,
            'exp' : years_of_exp,
            'job_level': job_level,
            'group_job':group_job_name,
            'job_type': job_type,
            'benefit': benefit_names,
            'job_des': job_description,
            'job_req': job_requirement,
            'city' : city,
            'address': address,
            'web':"VietNamWork"
            }
        
        pipeline.process_item(job_data)

        jobs_list.append(job_data)

    page += 1
    payload['page'] = page

In [ ]:
for job in jobs_list:
    if isinstance(job.get('time_update'), datetime):
        job['time_update'] = job['time_update'].strftime('%Y-%m-%d')
    if isinstance(job.get('time_expire'), datetime):
        job['time_expire'] = job['time_expire'].strftime('%Y-%m-%d')

# Specify the CSV file headers based on the keys in your job_data dictionary
csv_headers = [
    'id', 'url', 'title', 'company_name', 'company_url', 'time_update', 'time_expire',
    'salary', 'exp', 'job_level', 'group_job', 'job_type', 'benefit',
    'job_des', 'job_req', 'city', 'address', 'web'
]

# Save jobs_list to a CSV file
with open('vietnamwork.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=csv_headers)
    writer.writeheader()  # Write the header row
    writer.writerows(jobs_list)  # Write all job data rows